<!--COURSE_INFORMATION-->
<img align="left" style="padding-right:10px;" src="https://sitejerk.com/images/google-earth-logo-png-5.png" width=5% >
<img align="right" style="padding-left:10px;" src="https://colab.research.google.com/img/colab_favicon_256px.png" width=6% >


>> *This notebook is part of the free course [EEwPython](https://colab.research.google.com/github/csaybar/EEwPython/blob/master/index.ipynb); the content is available [on GitHub](https://github.com/csaybar/EEwPython)* and released under the [Apache 2.0 License](https://www.gnu.org/licenses/gpl-3.0.en.html). 99% of this material has been adapted from [Google Earth Engine Guides](https://developers.google.com/earth-engine/).

<!--NAVIGATION-->
| [Contents](index.ipynb) | [Image](2_eeImage.ipynb) >

<a href="https://colab.research.google.com/github/csaybar/EEwPython/blob/master/1_Introduction.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>


<center>
<h1>Google Earth Engine with Python </h1>
<h2> Developer's Guide </h2>
</center>
<h2> Topics:</h2>

1. What is Google Earth Engine?
2. Why python instead of code editor (Javascript)?
3. Connecting GEE with other Google Services
4. Hello World!
5. Fundamental Earth Engine data structures



### Authentication 

To read/write from a Google Drive or Google Cloud Storage bucket to which you have access, it's necessary to authenticate (as yourself). You'll also need to authenticate as yourself with Earth Engine, so that you'll have access to your scripts, assets, etc.

#### Authenticate to Earth Engine

In order to access Earth Engine, signup at [signup.earthengine.google.com](https://signup.earthengine.google.com/).

Identify yourself to Google Cloud, so you have access to storage and other resources. When you run the code below, it will display a link in the output to an authentication page in your browser. Follow the link to a page that will let you grant permission to the Google Earth Engine to access your resources. Copy the code from the permissions page back into this notebook and press return to complete the process.



In [4]:
import ee

In [10]:
ee.Authenticate()


Successfully saved authorization token.


 Once you have authorized access (just Earth Engine auth is required). Initialize your connection as shown the cell below.

In [11]:
ee.Initialize(project='ee-ivanlizarazo')

## 6. Hello World!


- **assetID**: Is the key to identify a specific file in [GEE datasets](https://developers.google.com/earth-engine/datasets/catalog/USGS_SRTMGL1_003).
- **ee.Image('assetID')**: In GEE raster data are represented as Image objects (see the next topic). 
- **ee.Image.clip(ee.Geometry.____)**: Is a ee.Image method used to crop spatially all the bands in Images.
- **folium**: bridge between Python and [Leaflet](https://leafletjs.com/)


In [13]:
import folium

Col = ee.Image("USGS/SRTMGL1_003")\
         .clip(ee.Geometry.Rectangle(-84, -4, -66,12))

# Define the URL format used for Earth Engine generated map tiles.
EE_TILES = 'https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}'

# Use folium to visualize the imagery.
mapid = Col.getMapId({'pallete': ['black', 'white'], 'min': 0, 'max': 3000})

map = folium.Map(location=[4.5, -75],zoom_start=5)
folium.TileLayer(
    tiles = mapid["tile_fetcher"].url_format,
    attr='Google Earth Engine',
    overlay=True,
    name='median composite',
  ).add_to(map)
map.add_child(folium.LayerControl())
map

## 7. Fundamental Earth Engine data structures

The two most fundamental geographic data structures in Earth Engine are **`Image`** and **`Feature`** corresponding to raster and vector data types, respectively. Images are composed of bands and a dictionary of properties. Features are composed of a **`Geometry`** and a dictionary of properties. A stack of images (e.g. an image time series) is handled by an **`ImageCollection`**. A collection of features is handled by a  **`FeatureCollection`**. Other fundamental data structures in Earth Engine include:

- `Dictionary`
- `List`
- `Array`
- `Geometry`
- `Date`
- `Number`
- `String`.

It is important to remember that these are all **server-side objects (or containers)**. That is, your client browser does not know anything about the objects in your script unless you explicitly request information about them. That request triggers a message being passed from Google to the Python API. If the message is large, there will be a corresponding slow down. Existing two ways to inspect **Earth Engine containers**  in Python console. The first one, `print()` that return the  petition (as a JSON) to the server and `*.getInfo()` that returns the contents of the `container`.  The next section summaries all the Data Structures mentioned above.



#### 7.1. Strings

Define a Python string, then put it into the **`ee.String()`** container to be sent to Earth Engine:

In [0]:
# Define a string, then put it into an EE container.
aString = 'To the cloud!'
eeString = ee.String(aString)
print('Where to?', eeString)
eeString

Think of **`ee.Thing`** as a container for a thing that exists on the server. In this example, the string is defined first, then put into the container. 

### 7.2. Numbers

Use **`ee.Number()`** to create number objects on the server.

In [0]:
# Define a number that exists on the server.
import numpy as np
serverNumber = ee.Number(np.e)
print('e=', serverNumber)
serverNumber

### 7.3. Lists

To make a Python list into an **`ee.List`** object on the server, you can put a Python literal into a container as with numbers and strings. Earth Engine also provides server-side convenience methods for making sequences of numbers.



In [0]:
# Make a sequence the hard way.
eeList = ee.List([1, 2, 3, 4, 5])

# Make a sequence the easy way!
sequence = ee.List.sequence(1, 5);

print('Sequence:', sequence)
print('Opening the container:', sequence.getInfo())

### 7.4. Dictionaries

You can construct an Earth Engine `Dictionary` from a Python object, as with strings, numbers and lists. At construction time, you can use Python functionality to initialize the Earth Engine object. In this case an ee.Dictionary is constructed directly from a Python literal object:

In [0]:
dictionary = ee.Dictionary({
  'e': np.e,
  'pi': np.pi,
  'phi': (1 + np.sqrt(5))/2
})

# Get some values from the dictionary.
print('Euler:', dictionary.get('e').getInfo())
print('Pi:', dictionary.get('pi').getInfo())
print('Golden ratio:', dictionary.get('phi').getInfo())

# Get all the keys:
print('Keys: ', dictionary.keys().getInfo())

In this example, observe that once you have an `ee.Dictionary`, you must use methods on the `ee.Dictionary` to get values. Specifically, get(key) returns the value associated with key. Since the type of object returned by `get()` could be be anything, if you're going to do anything to the object other then print it, you need to cast it to the right type. Also note that the `keys()` method returns an List.

### 7.5. Dates

Date objects are the way Earth Engine represents time. As in the previous examples, it is important to distinguish between a Python Date object and an Earth Engine `ee.Date` object.

A Date object can be construct from:
  - A string.
  - Python [datetime](https://docs.python.org/3/library/datetime.html) object.
  - Static methods provided by the ee.Date class.
  
The next examples illustrates the construction of dates from these three forms.

In [0]:
# Define a date in Earth Engine.
date = ee.Date('2015-12-31')
print('Date:', date)
print('Date:', date.getInfo())

In earth engine the date is representing with timestamps since midnight on January 1, 1970 considering an in milliseconds timestep . 

In [0]:
# Get the current time using the JavaScript Date.now() method.
import datetime as dt
now = dt.datetime.now()
print('Milliseconds since January 1, 1970', now)

# Initialize an ee.Date object.
eeNow = ee.Date(now)
print('Now:', eeNow)

Arguments to Earth Engine methods can be passed in order, for example to create an `ee.Date` from year, month and day, you can pass parameters of the `fromYMD()` static method in the order year, month, day:



In [0]:
aDate = ee.Date.fromYMD(2017, 1, 13)
print('aDate:', aDate)

<!--NAVIGATION-->
| [Contents](index.ipynb) | [Image](2_eeImage.ipynb) >

<a href="https://colab.research.google.com/github/csaybar/EEwPython/blob/master/1_Introduction.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>
